In [0]:
%pylab inline

In [1]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import requests
from bs4 import BeautifulSoup
import csv

In [2]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
place_code = {"東京":{"prec_no":"44", "block_no":"47662", "type":"s1"}, 
              "大分":{"prec_no":"83", "block_no":"0808", "type":"a1"}, 
              "弘前":{"prec_no":"31", "block_no":"0166", "type":"a1"}}
base_url = "http://www.data.jma.go.jp/obd/stats/etrn/view/daily_%s.php?prec_no=%s&block_no=%s&year=%s&month=%s&day=&view=p1"

In [8]:
# base_url = "http://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=83&block_no=0933&year=2011&month=12&day=&view=p1"

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
#取ったデータをfloat型に変えるやつ。(データが取れなかったとき気象庁は"/"を埋め込んでいるから0に変える)
def str2float(str):
    try:
        return float(str)
    except:
        return 0.0

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
l = []
#都市を網羅します
for place in place_code.keys():
    #最終的にデータを集めるリスト (下に書いてある初期値は一行目。つまり、ヘッダー。)
    # index = place_name.index(place)
    prec_no = place_code[place]["prec_no"]
    block_no = place_code[place]["block_no"]
    data_type = place_code[place]["type"]

    # for文で2007年~2017年までの11回。
    for year in range(2020, 2022):# range(1991, 2022):
        print(year)
        # その年の1月~12月の12回を網羅する。
        if year == 2021:
            last_month = 3
        else:
            last_month = 13
        for month in range(1,last_month):
            #print(month)
            #2つの都市コードと年と月を当てはめる。
            url = base_url%(data_type, prec_no, block_no, year, month)
            print(place, year, month, url)
            r = requests.get(url)
            r.encoding = r.apparent_encoding

            # まずはサイトごとスクレイピング
            soup = BeautifulSoup(r.text)
            # findAllで条件に一致するものをすべて抜き出します。
            # 今回の条件はtrタグでclassがmtxになってるものです。
            rows = soup.findAll('tr',class_='mtx')

            # 表の最初の1~4行目はカラム情報なのでスライスする。(indexだから初めは0だよ)
            # 【追記】2020/3/11 申し訳ございません。間違えてました。
            if data_type == "a1":
                rows_data = rows[3:]

                # 1日〜最終日までの１行を網羅し、取得します。
                for row in rows_data:
                    # 今度はtrのなかのtdをすべて抜き出します
                    data = row.findAll('td')

                    #１行の中には様々なデータがあるので全部取り出す。
                    # ★ポイント
                    day = data[0].findAll('a')[0].string # 日にち
                    date = str(year) + "/" + str(month) + "/" + day
                    precip_total  = data[1].string #降水量(mm)合計
                    temp_ave = data[4].string #平均気温
                    temp_high = data[5].string #最高気温
                    temp_low = data[6].string #最低気温
                    daylight = data[13].string #日照時間
                    d = {"date":date, "precip_total":precip_total, "temp_ave":temp_ave, "temp_high":temp_high, "temp_low":temp_low, "daylight":daylight, "place":place}
                    l.append(d)
            else:
                rows_data = rows[4:]
                for row in rows_data:
                    # 今度はtrのなかのtdをすべて抜き出します
                    data = row.findAll('td')

                    #１行の中には様々なデータがあるので全部取り出す。
                    # ★ポイント
                    day = data[0].findAll('a')[0].string
                    date = str(year) + "/" + str(month) + "/" + day
                    precip_total  = data[3].string #降水量(mm)合計
                    temp_ave = data[6].string #平均気温
                    temp_high = data[7].string #最高気温
                    temp_low = data[8].string #最低気温
                    daylight = data[16].string #日照時間
                    d = {"date":date, "precip_total":precip_total, "temp_ave":temp_ave, "temp_high":temp_high, "temp_low":temp_low, "daylight":daylight, "place":place}
                    l.append(d)

2020
東京 2020 1 http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2020&month=1&day=&view=p1
東京 2020 2 http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2020&month=2&day=&view=p1
東京 2020 3 http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2020&month=3&day=&view=p1
東京 2020 4 http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2020&month=4&day=&view=p1
東京 2020 5 http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2020&month=5&day=&view=p1
東京 2020 6 http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2020&month=6&day=&view=p1
東京 2020 7 http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2020&month=7&day=&view=p1
東京 2020 8 http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2020&month=8&

In [4]:
df = pd.DataFrame(l)

In [5]:
df

,date,daylight,place,precip_total,temp_ave,temp_high,temp_low
0,1991/1/1,--,東京,7.5,6.6,9.0,5.3
1,1991/1/2,0.1,東京,0.0,7.9,9.9,4.0
2,1991/1/3,1.3,東京,0.0,8.3,10.1,5.9
3,1991/1/4,8.2,東京,--,7.1,11.6,1.9
4,1991/1/5,7.6,東京,--,6.5,9.7,3.4
5,1991/1/6,7.9,東京,--,5.4,9.4,2.4
6,1991/1/7,9.0,東京,--,5.5,9.4,1.2
7,1991/1/8,7.4,東京,--,5.7,10.5,1.0
8,1991/1/9,9.4,東京,--,5.8,10.0,1.9
9,1991/1/10,9.1,東京,--,7.5,11.5,3.6
